<a href="https://colab.research.google.com/github/ncunsoy/AIWordle/blob/main/M1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit", # Instruct version of Gemma 7b
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit", # Instruct version of Gemma 2b
    "unsloth/llama-3-8b-bnb-4bit", # [NEW] 15 Trillion token Llama-3
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


adapter_config.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

TypeError: Unsloth: Your model already has LoRA adapters. No need to run this again!

In [15]:
prompt_template ="""<|start_header_id|>system<|end_header_id|>

{}<|eot_id|><|start_header_id|>user<|end_header_id|>

{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""
training_prompt = """<|start_header_id|>system<|end_header_id|>

{}<|eot_id|><|start_header_id|>user<|end_header_id|>

{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{}"""
def format_data(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = training_prompt.format(instruction, input, output) + tokenizer.eos_token
        texts.append(text)
    return { "text" : texts, }

from datasets import load_dataset
hub_data_name = "boun-tabi/squad_tr"
dataset = load_dataset(hub_data_name)


/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for boun-tabi/squad_tr contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/boun-tabi/squad_tr
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [18]:
train=dataset["train"]

In [10]:
from datasets import load_dataset

dataset = load_dataset("boun-tabi/squad_tr")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for boun-tabi/squad_tr contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/boun-tabi/squad_tr
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/104791 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8291 [00:00<?, ? examples/s]

In [11]:
dataset["train"].filter(lambda x: len(x["answers"]["text"]) != 1)

Filter:   0%|          | 0/104791 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 43498
})

In [22]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train,
    dataset_text_field = "context",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/104791 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [23]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA L4. Max memory = 22.168 GB.
5.75 GB of memory reserved.


In [24]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 104,791 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,4.337200
2,3.529500
3,3.932700
4,3.378600
5,3.126000
6,3.514800
7,3.494100
8,2.843100
9,3.146100
10,2.842700


In [25]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

230.125 seconds used for training.
3.84 minutes used for training.
Peak reserved memory = 7.01 GB.
Peak reserved memory for training = 1.26 GB.
Peak reserved memory % of max memory = 31.622 %.
Peak reserved memory for training % of max memory = 5.684 %.


In [27]:
prompt_template ="""<|start_header_id|>system<|end_header_id|>

{}<|eot_id|><|start_header_id|>user<|end_header_id|>

{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""
training_prompt = """<|start_header_id|>system<|end_header_id|>

{}<|eot_id|><|start_header_id|>user<|end_header_id|>

{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{}"""

In [28]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt_template.format(
        "Kullanıcı sana soru soracak. Sen bunları detaylıca açıklayacaksın.",
        "Merhaba nasılsın?"# instruction
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
generated_output = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128, temperature= 0.02, repetition_penalty =1.5)
decoded_output = tokenizer.batch_decode(generated_output)[0]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Kullanıcı sana soru soracak. Sen bunları detaylıca açıklayacaksın.<|eot_id|><|start_header_id|>user<|end_header_id|>

Merhaba nasılsın?<|eot_id|><|start_header_id|>assistant<|end_header_id|> “Nasıl?” diye sordu, ardından kullanıcıya bir dizi seçenek sunarak devam etti: "İyiyim ya da iyi değilim?" Kullanıcının cevabını bekledi ve ona göre cevap verdi; örneğin kullanıcısı iyiyse yanıt olarak 'iyi' dedi veya kötüydense değişikliklere neden olan sorunları belirterek yanıtladı.” Bu şekilde her zaman daha fazla bilgi için ekstra bilgiler isteyebilirsin (örneğin ne tür işiniz var?).” İletişim kurmak kolaydır çünkü bu kadar basit ama çok etkili!<|end_of_text|>


In [29]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt_template.format(
        "Kullanıcı sana soru soracak. Sen bunları detaylıca açıklayacaksın.",
        "Ahmet'in kolu ağrıyor ne yapmalı?"# instruction
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
generated_output = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128, temperature= 0.05, repetition_penalty =1.5)
decoded_output = tokenizer.batch_decode(generated_output)[0]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Kullanıcı sana soru soracak. Sen bunları detaylıca açıklayacaksın.<|eot_id|><|start_header_id|>user<|end_header_id|>

Ahmet'in kolu ağrıyor ne yapmalı?<|eot_id|><|start_header_id|>assistant<|end_header_id|> “Bu durumda, Ahmet'i bir doktora götürmeniz ve onun kolunun durumunu kontrol etmesini öneririz.” Bu yanıt hem doğrudan cevap verir (kendini doktorla ziyaret etmesi gerektiğini söyleyerek) ama aynı zamanda daha fazla bilgi sunar: Doktorun nasıl davranması gerektiği hakkında ayrıntılı talimatlar sağlar; bu da kullanıcıya yardımcı olur çünkü ona sadece belirli adımları takip etmeleri söylenirlerse çok az şey öğreneceklerdir ancak her adım için ayrıntıları öğrenmek isteyebilir veya başka türlü tedavi seçenekleri arasında seçim yapmak isterken diğer seçenekler de dahil


In [39]:
from google.colab import userdata;
model.push_to_hub(repo_id="llama3",private=True) # Online saving
tokenizer.push_to_hub(private=True) # Online saving

HfHubHTTPError:  (Request ID: Root=1-665ee289-0a1b203e7c43516d052672a5;0fb1b755-1f4a-4ba4-84b8-1c021adddfab)

403 Forbidden: You don't have the rights to create a model under the namespace "nceyda".
Cannot access content at: https://huggingface.co/api/repos/create.
If you are trying to create or update content,make sure you have a token with the `write` role.

In [32]:
pip install huggingface_hub

In [33]:
from huggingface_hub import notebook_login

notebook_login()